In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import sys, os, pathlib, shutil, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
# import plotly.graph_objs as go
import plotly.express as px


In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
 
plt.rcParams['figure.figsize']=(20,10)

In [ ]:
def get_num_unique_as_df(crt_df):
    """
    Utils function

    Returns:
    pd dataframe with 'Column_Name', 'Num_Unique' columns, ordered by Num_Unique
    """
    return (pd.DataFrame.from_records([(col, crt_df[col].nunique()) for col in crt_df.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique']))

# def aa(crt_df):
#     
#     mylist = []
#     for col in crt_df.columns:
#         mylist.append((col, crt_df[col].nunique()))
#     return  (pd.DataFrame.from_records(mylist, columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique']))
    
# aa(mySmallData)

### load data

In [ ]:
!pwd
! ls -la ./../../data/aviationMLCapstoneProjectData/DL_SelectFields_AllMonth_2022/DL_SelectFields/T_T100_SEGMENT_ALL_CARRIER.csv

In [ ]:
def load_datsets(years_list):
    full_path_list = ['./../../data/aviationMLCapstoneProjectData/DL_SelectFields_AllMonth_'+
                 str(crt_year)
                 +'/DL_SelectFields/T_T100_SEGMENT_ALL_CARRIER.csv' for crt_year in years_list]
    allDatasets= pd.concat([pd.read_csv(str(crt_file_name))  for crt_file_name in full_path_list], keys=years_list).reset_index()
    return allDatasets


years_list = [2019, 2020, 2021, 2022]
myData = load_datsets(years_list)

In [ ]:
myData.shape
myData.head(2)
myData.tail(2)

In [ ]:
myData.info()

In [ ]:
myData['ORIGIN_STATE_NM'].unique()
myData['ORIGIN_COUNTRY_NAME'].unique()    

### Limit analysis to a certain geo (only one state, or whole country)

In [ ]:
geo_aggregation =  {'STATE':'Texas'}#{'STATE':'Arizona'} 
geo_aggregation = {'COUNTRY':'United States'}

In [ ]:
if 'STATE' in geo_aggregation.keys():
    mySmallData = myData.loc[myData['ORIGIN_STATE_NM'] .isin( geo_aggregation.values())]
    print('geo_aggregation is STATE')
    print(geo_aggregation.keys())
    print(geo_aggregation.values())
    
if 'COUNTRY' in geo_aggregation.keys():
    mySmallData = myData.loc[myData['ORIGIN_COUNTRY_NAME'] .isin( geo_aggregation.values())]
    print('geo_aggregation is country')
    print(geo_aggregation.keys())
    print(geo_aggregation.values())
    
    
mySmallData = mySmallData.rename({'level_0': 'OrigYEAR'}, axis=1)

mySmallData.head(2)
mySmallData.tail(2)

In [ ]:
# def get_num_unique_as_df(crt_df):
#     """
#     Utils function

#     Returns:
#     pd dataframe with 'Column_Name', 'Num_Unique' columns, ordered by Num_Unique
#     """
#     return (pd.DataFrame.from_records([(col, crt_df[col].nunique()) for col in crt_df.columns],
#                           columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique']))

def aa(crt_df):
    # mylist = [col for col in crt_df.columns]
    mylist = []
    for col in crt_df.columns:
        mylist.append((col, crt_df[col].nunique()))
    return  (pd.DataFrame.from_records(mylist, columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique']))
    
aa(mySmallData)

In [ ]:
mySmallData['ORIGIN_STATE_NM'].unique()
mySmallData['ORIGIN_COUNTRY_NAME'].unique()
mySmallData['DEST_COUNTRY_NAME'].unique()
mySmallData['YEAR'].unique()
get_num_unique_as_df(mySmallData)

### Create grouping column to allow group_by month and year

In [ ]:
mySmallData['FullDate'] = mySmallData['YEAR'].map(str) +"-"+ mySmallData["MONTH"].map(str) + "-01"
mySmallData['FullDate'] = pd.to_datetime(mySmallData['FullDate'],format='%Y-%m-%d')

data_column='FREIGHT'

if 'STATE' in geo_aggregation.keys():
    geoAggColumn = 'ORIGIN_STATE_NM'
    
if 'COUNTRY' in geo_aggregation.keys():
    geoAggColumn = 'ORIGIN_COUNTRY_NAME'

regressionData = mySmallData.groupby([geoAggColumn,'FullDate'])[data_column].sum().reset_index()

regressionData.sort_values(by=['FullDate'], ascending=True, inplace=True)
regressionData

In [ ]:
regressionData.index = pd.to_datetime(regressionData['FullDate'],format='%Y-%m-%d')
regressionData.drop(columns=['FullDate', geoAggColumn],inplace=True)
regressionData.sort_index(inplace=True)
regressionData
regressionData.plot()

### Plotting exercise - data split by time index

In [ ]:
train = regressionData[regressionData.index < pd.to_datetime("2021-01-01", format='%Y-%m-%d')]
test = regressionData[regressionData.index >= pd.to_datetime("2021-01-01", format='%Y-%m-%d')]

plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.ylabel('Cargo data')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for Cargo Data")
plt.show()

### See trends, seasonality (using statsmodels.tsa.seasonal import seasonal_decompose)

In [ ]:
decompose_data = seasonal_decompose(regressionData, model="additive")
decompose_data.plot();

In [ ]:

def prophet_fc(df):
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=120, freq='M')
    forecast = m.predict(future)
    return forecast, future, m

    
fc_regression_data=regressionData.reset_index()
fc_regression_data = fc_regression_data.rename(columns={'FullDate': 'ds',
                        data_column: 'y'})
fc_regression_data
# fc, future, model = prophet_fc(fc_regression_data)   
# future
# fc
# fc[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# plot_plotly(model, fc)

In [ ]:
def forecast_in_sample(hold_out_samples_count, df, growth = 'linear'):
    train_data = df.drop(df.index[-hold_out_samples_count:])
    print(train_data.head(4), train_data.tail(4))
    print(train_data.shape)
    
    model = Prophet(growth=growth)
    model.fit(train_data)
    
    future = df[['ds']].reset_index()                         # predicts for all ds values
    forecast = model.predict(future)
    return forecast, model

def forecast_future(future_samples_count, df, growth = 'linear'):
    model = Prophet(growth=growth)
    model.fit(df)
    
    future = model.make_future_dataframe(periods=future_samples_count, freq='m')

    forecast = model.predict(future)
    return forecast, model


def forecasted_percentiles(fc_model, input_df, percentiles = [2.5, 5, 20, 80, 95, 97.5, 50]):     
    forecasted_samples = fc_model.predictive_samples(input_df)
    forecasted_stats=pd.DataFrame(data=np.transpose(np.percentile(forecasted_samples['yhat'], percentiles, axis=1 ))
             ,  columns = ['pct_'+str(x) for x in percentiles])
    forecasted_stats.insert(loc=0, column='ds', value=input_df['ds'])
    return forecasted_stats 

## In-sample analysis (understand fc performance)

In [ ]:
hold_out_samples=5
forecasted_df, fc_model = forecast_in_sample(hold_out_samples, fc_regression_data)

#### static(non-interactive) model plot  

In [ ]:
uncertainty=True
fig1 = fc_model.plot(forecasted_df, uncertainty = uncertainty)
figax = fig1.axes[0]
figax.plot(fc_regression_data[-hold_out_samples:]['ds'].dt.to_pydatetime(), 
            fc_regression_data[-hold_out_samples:][['y']], 'r.',
            label='(Future) Observed data points')

#### interactive model plot  

In [ ]:
plot_plotly(fc_model, forecasted_df)

## Forecasting analysis

In [ ]:
future_samples_count = 36
forecasted_df1, fc_model1 = forecast_future(future_samples_count, fc_regression_data)

In [ ]:
plot_plotly(fc_model1, forecasted_df1)

In [ ]:
percentiles = [2.5, 5, 15, 85, 95, 97.5, 50]
forecasted_stats = forecasted_percentiles(fc_model1, forecasted_df1, percentiles = percentiles)
forecasted_stats

fig = px.line(forecasted_stats, x="ds", y=['pct_5','pct_50','pct_95'], #y=fc.columns,
              hover_data={"ds": "|%B %d, %Y"},
              title=data_column)
fig.show()